In [2]:
# some useful mysklearn package import statements and reloads
import importlib

import pandas as pd
import numpy as np
from collections import Counter
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder


# ✈️ Flight Delay Classification ✈️

In [3]:
import pandas as pd
from sklearn.model_selection import KFold

# Load data
data = pd.read_csv('balanced_flights.csv')

# Compute flight delay (in minutes, assuming sched_dep_time and dep_time are timestamps or integers)
data['flight_delay'] = data['sched_dep_time'] - data['dep_time']

# Extract meaningful features for predicting delays
X = data[[
    'year', 'month', 'day', 'hour', 'minute',  # Time-based features
    'carrier', 'flight', 'tailnum',            # Operational features
    'origin', 'dest', 'distance', 'air_time', # Location & flight characteristics
    'dep_delay', 'sched_dep_time'             # Existing delay and timing
]]

# Target variable: flight delay
y = data['flight_delay']

# Initialize K-Fold cross-validation (10 splits)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Placeholder for performance metrics
accuracies, error_rates, precisions, recalls, f1_scores, confusion_matrices = [], [], [], [], [], []


In [4]:
import numpy as np
import pandas as pd

# Load the dataset (assuming `data` is a pandas DataFrame containing the provided sample data)
data = pd.read_csv('balanced_flights.csv') 

# Define delay categories
def categorize_delay(delay):
    if delay <= 0:
        return 0
    elif 0 < delay <= 30:
        return 1
    elif 30 < delay <= 60:
        return 2
    elif 60 < delay <= 120:
        return 3
    elif 120 < delay <= 180:
        return 4
    else:
        return 5

# Map delays to categories
data['delay_category'] = data['dep_delay'].apply(categorize_delay)

# Convert categorical features to integers (label encoding)
categorical_cols = ['carrier', 'flight', 'tailnum', 'origin', 'dest']  # List your categorical columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))  # Convert to string if necessary and apply label encoding
    label_encoders[col] = le  # Store the encoder to use it later if needed

# Prepare features (X) and labels (y)
X = data[['dep_time', 'sched_dep_time', 'sched_arr_time', 'air_time', 'dest',
          'carrier', 'hour']].fillna(0)

y = data['delay_category'].values

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


### kNN Performance Evaluation & Confusion Matrix

In [6]:
import numpy as np
import pandas as pd

X = np.array(X) 
y = np.array(y) 

# Initialize StratifiedKFold with k=10 (assuming myevaluation provides this utility)
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics (accuracy, precision, recall, f1, confusion matrices, etc.)
knn_accuracies, knn_precisions, knn_recalls, knn_f1s, knn_conf_matrices = [], [], [], [], []

# Initialize lists to gather true and predicted values across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold
for train_index, test_index in kf:
    knn_X_train, knn_X_test = X[train_index], X[test_index]
    knn_y_train, knn_y_test = y[train_index], y[test_index]

    # Initialize and fit MyKNeighborsClassifier
    knn_model = MyKNeighborsClassifier()
    knn_model.fit(knn_X_train.tolist(), knn_y_train.tolist())
    
    # Make predictions on the test set
    knn_y_pred = knn_model.predict(knn_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(knn_y_test, knn_y_pred)
    precision_score = myevaluation.binary_precision_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class precision
    recall_score = myevaluation.binary_recall_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class recall
    f1 = myevaluation.binary_f1_score(knn_y_test, knn_y_pred, labels= labels)  # Multi-class F1 score
    confusion_matrix = myevaluation.confusion_matrix(knn_y_test, knn_y_pred, labels=labels)

    # Store fold metrics
    knn_accuracies.append(accuracy)
    knn_precisions.append(precision_score)
    knn_recalls.append(recall_score)
    knn_f1s.append(f1)
    knn_conf_matrices.append(confusion_matrix)
    
    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(knn_y_test)
    all_y_pred.extend(knn_y_pred)

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

ValueError: pos_label must be specified for binary classification.

In [8]:
print("===========================================")
print("kNN Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(knn_accuracies):.2f}")
print(f"Error: {1 - np.mean(knn_accuracies):.2f}")
print(f"Precision: {np.mean(knn_precisions):.2f}")
print(f"Recall: {np.mean(knn_recalls):.2f}")
print(f"F1 Score: {np.mean(knn_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

kNN Classifier Performance
Accuracy: 0.63
Error: 0.37
Precision: 0.58
Recall: 0.68
F1 Score: 0.62

Confusion Matrix:
  Delayed    0    1    2    3    4    5    Total    Recognition %
---------  ---  ---  ---  ---  ---  ---  -------  ---------------
        0  204   89    6    1    0    0      300            68.00
        1  118  140   39    3    0    0      300            46.67
        2   30   80  142   46    2    0      300            47.33
        3    3   12   59  187   39    0      300            62.33
        4    0    0    2   51  209   38      300            69.67
        5    0    0    1    5   47  247      300            82.33


### Naive Bayes Evaluation & Confusion Matrix

In [1]:
import numpy as np
import pandas as pd

X = np.array(X) 
y = np.array(y) 

# Initialize StratifiedKFold with k=10
kf = myevaluation.stratified_kfold_split(X, y, n_splits=10, random_state=None, shuffle=False)

# Initialize lists to store metrics
nb_accuracies, nb_precisions, nb_recalls, nb_f1s, nb_conf_matrices = [], [], [], [], []

# Initialize lists to collect all true and predicted labels across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold in Stratified K-Fold
for train_index, test_index in kf:
    nb_X_train, nb_X_test = X[train_index], X[test_index]
    nb_y_train, nb_y_test = y[train_index], y[test_index]

    # Initialize and fit MyNaiveBayesClassifier
    nb_model = MyNaiveBayesClassifier()
    nb_model.fit(nb_X_train.tolist(), nb_y_train.tolist())
    
    # Make predictions on the test set
    nb_y_pred = nb_model.predict(nb_X_test.tolist())
    
    # Calculate and store metrics for this fold
    accuracy = myevaluation.accuracy_score(nb_y_test, nb_y_pred)
    error = 1 - accuracy
    precision_score = myevaluation.binary_precision_score(nb_y_test, nb_y_pred, labels=labels)
    recall_score = myevaluation.binary_recall_score(nb_y_test, nb_y_pred, labels=labels)
    f1 = myevaluation.binary_f1_score(nb_y_test, nb_y_pred, labels=labels)
    confusion_matrix = myevaluation.confusion_matrix(nb_y_test, nb_y_pred, labels=labels)

    # Store metrics for this fold
    nb_accuracies.append(accuracy)
    nb_precisions.append(precision_score)
    nb_recalls.append(recall_score)
    nb_f1s.append(f1)
    nb_conf_matrices.append(confusion_matrix)

    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(nb_y_test)
    all_y_pred.extend(nb_y_pred)
   

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels=labels)
matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

NameError: name 'X' is not defined

In [11]:
print("===========================================")
print("Naive Bayes Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(nb_accuracies):.2f}")
print(f"Error: {1 - np.mean(nb_accuracies):.2f}")
print(f"Precision: {np.mean(nb_precisions):.2f}")
print(f"Recall: {np.mean(nb_recalls):.2f}")
print(f"F1 Score: {np.mean(nb_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

Naive Bayes Classifier Performance
Accuracy: 0.22
Error: 0.78
Precision: 0.37
Recall: 0.36
F1 Score: 0.36

Confusion Matrix:
  Delayed    0    1    2    3    4    5    Total    Recognition %
---------  ---  ---  ---  ---  ---  ---  -------  ---------------
        0  109   45   44   32   30   40      300            36.33
        1   43   67   51   54   52   33      300            22.33
        2   49   51   46   66   48   40      300            15.33
        3   35   49   53   52   51   60      300            17.33
        4   40   50   36   55   53   66      300            17.67
        5   24   39   46   51   73   67      300            22.33


### Random Forest Performance Evaluation & Confusion Matrix

In [12]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold

X = np.array(X)
y = np.array(y)

# Initialize StratifiedKFold with k=10
kf = StratifiedKFold(n_splits=10, shuffle=False, random_state=None)

# Initialize lists to store metrics
rf_accuracies, rf_precisions, rf_recalls, rf_f1s, rf_conf_matrices = [], [], [], [], []

# Initialize lists to collect all true and predicted labels across all folds
all_y_true = []
all_y_pred = []

# Define the correct labels for the confusion matrix and metrics
labels = [0, 1, 2, 3, 4, 5]

# Loop over each fold
for train_index, test_index in kf.split(X, y):
    rf_X_train, rf_X_test = X[train_index], X[test_index]
    rf_y_train, rf_y_test = y[train_index], y[test_index]

    # Initialize and fit RandomForestClassifier
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(rf_X_train, rf_y_train)

    # Make predictions on the test set
    rf_y_pred = rf_model.predict(rf_X_test)

    # Calculate and store metrics for this fold
    accuracy = accuracy_score(rf_y_test, rf_y_pred)
    precision = precision_score(rf_y_test, rf_y_pred, labels=labels, average='macro', zero_division=0)
    recall = recall_score(rf_y_test, rf_y_pred, labels=labels, average='macro', zero_division=0)
    f1 = f1_score(rf_y_test, rf_y_pred, labels=labels, average='macro', zero_division=0)
    conf_matrix = confusion_matrix(rf_y_test, rf_y_pred, labels=labels)

    # Store metrics for this fold
    rf_accuracies.append(accuracy)
    rf_precisions.append(precision)
    rf_recalls.append(recall)
    rf_f1s.append(f1)
    rf_conf_matrices.append(conf_matrix)

    # Collect all true and predicted labels for final confusion matrix
    all_y_true.extend(rf_y_test)
    all_y_pred.extend(rf_y_pred)

# Calculate the final confusion matrix using all folds' predictions
final_confusion_matrix = confusion_matrix(all_y_true, all_y_pred, labels=labels)

# Optionally add functionality to calculate totals for the confusion matrix if required
# matrix_with_totals = myutils.calculate_confusion_matrix_totals(final_confusion_matrix)

# Print or return the results as needed
print("Accuracies:", rf_accuracies)
print("Precisions:", rf_precisions)
print("Recalls:", rf_recalls)
print("F1 Scores:", rf_f1s)
print("Final Confusion Matrix:\n", final_confusion_matrix)


Accuracies: [0.6111111111111112, 0.6333333333333333, 0.6611111111111111, 0.6722222222222223, 0.6333333333333333, 0.6111111111111112, 0.6, 0.6722222222222223, 0.5777777777777777, 0.5944444444444444]
Precisions: [0.6284058501725535, 0.6380461406045388, 0.6663845141786318, 0.6757498404479456, 0.6425925925925926, 0.619128602286497, 0.6045828159055819, 0.676750700280112, 0.5832846804111173, 0.5910270085916638]
Recalls: [0.6111111111111112, 0.6333333333333333, 0.6611111111111111, 0.6722222222222222, 0.6333333333333334, 0.6111111111111112, 0.6, 0.6722222222222222, 0.5777777777777778, 0.5944444444444444]
F1 Scores: [0.6136032350098509, 0.6334824955827353, 0.6604614162444622, 0.6701156689286204, 0.6301971326164875, 0.6106324405074463, 0.5975475147327222, 0.6727905100695136, 0.5797128707215145, 0.5921588568346935]
Final Confusion Matrix:
 [[183  89  26   2   0   0]
 [ 86 146  56  12   0   0]
 [ 35  70 142  48   5   0]
 [  7  10  49 180  53   1]
 [  0   0   2  44 225  29]
 [  1   0   0   4  43 25

In [13]:
print("===========================================")
print("Random Forest Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(rf_accuracies):.2f}")
print(f"Error: {1 - np.mean(rf_accuracies):.2f}")
print(f"Precision: {np.mean(rf_precisions):.2f}")
print(f"Recall: {np.mean(rf_recalls):.2f}")
print(f"F1 Score: {np.mean(rf_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

Random Forest Classifier Performance
Accuracy: 0.63
Error: 0.37
Precision: 0.63
Recall: 0.63
F1 Score: 0.63

Confusion Matrix:
  Delayed    0    1    2    3    4    5    Total    Recognition %
---------  ---  ---  ---  ---  ---  ---  -------  ---------------
        0  109   45   44   32   30   40      300            36.33
        1   43   67   51   54   52   33      300            22.33
        2   49   51   46   66   48   40      300            15.33
        3   35   49   53   52   51   60      300            17.33
        4   40   50   36   55   53   66      300            17.67
        5   24   39   46   51   73   67      300            22.33


In [15]:
# Convert features and labels to numpy arrays
X = np.array(X)
y = np.array(y)

# Perform manual stratified K-fold
n_splits = 10
indices = np.arange(len(y))
unique_classes, y_counts = np.unique(y, return_counts=True)
folds = {i: [] for i in range(n_splits)}

for cls in unique_classes:
    cls_indices = indices[y == cls]
    np.random.shuffle(cls_indices)
    for i, index in enumerate(cls_indices):
        folds[i % n_splits].append(index)

fold_indices = [np.array(folds[i]) for i in range(n_splits)]

# Metrics storage
rf_accuracies, rf_precisions, rf_recalls, rf_f1s = [], [], [], []

# Iterate through folds
for i in range(n_splits):
    test_indices = fold_indices[i]
    train_indices = np.concatenate([fold_indices[j] for j in range(n_splits) if j != i])

    rf_X_train, rf_X_test = X[train_indices], X[test_indices]
    rf_y_train, rf_y_test = y[train_indices], y[test_indices]

    # Train Random Forest
    rf_model = MyRandomForestClassifier(n_estimators=1, max_depth=1)
    rf_model.fit(rf_X_train, rf_y_train)

    # Predictions
    rf_y_pred = rf_model.predict(rf_X_test)

    # Compute metrics
    accuracy = np.sum(rf_y_test == rf_y_pred) / len(rf_y_test)
    precision = myevaluation.binary_precision_score(rf_y_test, rf_y_pred)
    recall = myevaluation.binary_recall_score(rf_y_test, rf_y_pred)
    f1 = myevaluation.binary_f1_score(rf_y_test, rf_y_pred)

    # Store results
    rf_accuracies.append(accuracy)
    rf_precisions.append(precision)
    rf_recalls.append(recall)
    rf_f1s.append(f1)

    # Debugging outputs
    print(f"Fold {i + 1}")
    print("True Labels:", rf_y_test)
    print("Predicted Labels:", rf_y_pred)

# Print final metrics
print("Accuracies:", rf_accuracies)
print("Precisions:", rf_precisions)
print("Recalls:", rf_recalls)
print("F1 Scores:", rf_f1s)


Fold 1
True Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
Predicted Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 4, 4, 0, 0, 4, 4, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 4, 4, 0, 0, 0, 4, 4, 0, 0, 4, 0, 0, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 4, 4, 0, 4, 0, 4, 0, 0, 4]
Fold 2
True Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
print("===========================================")
print("Random Forest Classifier Performance")
print("===========================================")

# Calculate and print average performance
print(f"Accuracy: {np.mean(rf_accuracies):.2f}")
print(f"Error: {1 - np.mean(rf_accuracies):.2f}")
print(f"Precision: {np.mean(rf_precisions):.2f}")
print(f"Recall: {np.mean(rf_recalls):.2f}")
print(f"F1 Score: {np.mean(rf_f1s):.2f}\n")

# Display the confusion matrix
print("Confusion Matrix:")
myutils.display_confusion_matrix(matrix_with_totals, labels)

Random Forest Classifier Performance
Accuracy: 0.97
Error: 0.03
Precision: 0.96
Recall: 1.00
F1 Score: 0.98

Confusion Matrix:
  Delayed    0    1    2    3    4    5    Total    Recognition %
---------  ---  ---  ---  ---  ---  ---  -------  ---------------
        0   39   10    4    5    4    4       66            59.09
        1   12   15    9    9    2    1       48            31.25
        2    4   10   14    3    2    4       37            37.84
        3    6   10    1   21   10    9       57            36.84
        4   10    2    5    9   11    8       45            24.44
        5   11    1    5   11    8   11       47            23.40
